In [ ]:
import requests
import json
from IPython.display import JSON
from keys import *
#este archivo contiene las llaves de las APIs

In [ ]:
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/MMGL/2025-09-01T08:00/2025-09-01T20:00"
#url con la que tenemos que poner el aeropuerto y el rango de horas
#MMGL es el ICAO del aeropuerto de Guadalajara, Jalisco, México

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}
#parámetros que se le pueden agregar a la url
headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
}
response = requests.request("GET", url, headers=headers, params=querystring)
#hacemos la petición a la API

In [ ]:
if response.status_code == 200:
    new_data = response.json()

    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    existing_data.append(new_data)

    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)
#mostramos el código de estado para ver si la petición fue exitosa

Data appended to data.json file.


In [ ]:
flight_arrivals = response.json()

for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']

#se imprimiran los datos que nos interesan de los vuelos que llegan al aeropuerto

In [ ]:
import pandas as pd
from numpy import nan
#datos de los vuelos 
flight_arrivals = response.json()

def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'],
        'flight_number': flights['number'],
        'from' : flights['departure']['airport']['name'],
        'airline' : flights['airline']['name'],
        'aircraft' : flights['aircraft']['model']
    })
    return(flights_df)

flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

flight_arrivals.head()
#mostramos los datos en un dataframe de pandas

,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,AM 204,Mexico City,Aeromexico,Boeing 737-800,2025-09-01 14:30Z,2025-09-01 08:30-06:00
0,Y4 1001,Tijuana,Volaris,Airbus A320,2025-09-01 14:35Z,2025-09-01 08:35-06:00
0,Y4 152,Mexico City,Volaris,Airbus A320,2025-09-01 15:03Z,2025-09-01 09:03-06:00
0,AM 208,Mexico City,Aeromexico,Boeing 737,2025-09-01 15:08Z,2025-09-01 09:08-06:00
0,VB 3243,Puebla City,VivaAerobus,Airbus A320,2025-09-01 15:20Z,2025-09-01 09:20-06:00
